#### ACTUALIZAR MES EN EL fac_outstanding_monthly_snapshot
EL MES COLOCADO ES EL ÚNICO QUE SE ACTUALIZA, DEJA IGUAL TODOS LOS DEMÁS, pero ajusta nombres de proveedores y deudores

In [1]:
#%% Corte mensual actual
mes_incorporar     = '2025-12-31' # último día del mes, cambiar en cada ejecución
incluir_mes_actual = True # True para incluir el mes a incorporar, false para actualizar solo lo de atrás
cargar_sql         = True # True para que el resultado se cargue al sql, false para solo generar excels y dataframes

In [2]:
from datetime import datetime
hoy_formateado = datetime.today().strftime('%Y-%m-%d')

In [3]:
inicio = datetime.now()
print("Inicio:", inicio.strftime("%Y-%m-%d %H:%M:%S"))

import pandas as pd
!pip install boto3
import boto3
import json
import io
import os
import numpy as np
!pip install pyathena
from pyathena import connect

from google.colab import drive
drive.mount('/content/drive')

hoy_formateado = datetime.today().strftime('%Y-%m-%d')

Inicio: 2026-01-14 20:48:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.0 MB/s eta 0:00:00
Mounted at /content/drive


In [4]:
txt_credenciales_athena = r"credenciales actualizado.txt" # arrastrar json de credenciales

path = '/content/drive/MyDrive/BUSINESS ANALYTICS/FACTORING/PROYECTOS/fac_outstanding backups'

In [5]:
#%% funciones de transformación de fechas
mes_incorporar = pd.Timestamp(mes_incorporar)

def eomonth(fecha):
    """
    Devuelve el último día del mes de la fecha dada.
    """
    fecha = pd.Timestamp(fecha)
    return fecha + pd.offsets.MonthEnd(0)

def prev_month_eomonth(fecha):
    """
    Devuelve el último día del mes anterior a la fecha dada.
    """
    fecha = pd.Timestamp(fecha)  # asegurar tipo Timestamp
    fech2 = (fecha - pd.offsets.MonthBegin(1)) - pd.offsets.Day(1)
    return pd.Timestamp(fech2)

def convertir_codmes(fecha):
    fecha = pd.Timestamp(mes_incorporar)
    yyyymm = fecha.strftime("%Y%m")
    return yyyymm

eo_mes_actual   = eomonth(mes_incorporar)
eo_mes_anterior = prev_month_eomonth(mes_incorporar)
codmes          = convertir_codmes(mes_incorporar)
print(codmes)

202512


In [6]:
#%%
# Crea la carpeta si no existe
os.chdir(path)

folder_name = codmes
os.makedirs(folder_name, exist_ok=True)

os.chdir(path + '/' + folder_name)

In [7]:
#%% Credenciales de AmazonAthena
with open('/content/' + txt_credenciales_athena) as f:
    creds = json.load(f)

conn = connect(
    aws_access_key_id     = creds["AccessKeyId"],
    aws_secret_access_key = creds["SecretAccessKey"],
    aws_session_token     = creds["SessionToken"],
    s3_staging_dir        = creds["s3_staging_dir"],
    region_name           = creds["region_name"]

    )

#%% lectura del corte anterior

query = ''' select * from prod_datalake_master."ba__fac_outstanding_monthly_snapshot"  '''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_corte = pd.DataFrame(resultados, columns = column_names)

df_corte.to_excel(f'extraido {hoy_formateado}.xlsx', index = False)

del df_corte['_timestamp']

print(df_corte.shape)

(101731, 90)


In [8]:
#%% mes actual que vamos a incorporar

query = ''' select * from prod_datalake_analytics."fac_outst_unidos_f_desembolso_jmontoya"  '''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_view = pd.DataFrame(resultados, columns = column_names)
print(df_view.shape)

df_view.to_excel(f'fac_outstanding_tiempo_real_{codmes} - {hoy_formateado}.xlsx',
                  index = False)

df_view_completo = df_view.copy()

(106351, 90)


In [9]:
#%% agregando el mes actual
df_corte['codmes'] = df_corte['codmes'].astype(int)
df_corte = df_corte[ df_corte['codmes'] < int(codmes) ] #fitrando el mes actual en caso de que quisieramos añadir datos nuevamente

df_view['codmes'] = df_view['codmes'].astype(int)
df_view  = df_view[ df_view['codmes'] == int(codmes)]

#ASEGURANDO EL ORDENAMIENTO, PUES EXISTEN DIFERENCIAS POR MAYÚSCULAS Y MINÚSCULAS
columnas = ['code', 'fecha_cierre', 'codmes', 'product_type', 'client_ruc', 'client_name', 'provider_ruc', 'provider_name', 'flag_newclient', 'flag_newprovider', 'transfer_date', 'ANTERIOR_TRANSFER', 'currency_request', 'currency_auctions', 'assigned_financing_rate', 'total_net_amount_pending_payment', 'e_payment_date_original', 'amount_financed', 'terms', 'amount_advance', 'advance_percentage', 'invoice_count', 'amount_of_invoices', 'assigned_name', 'assigned_last_name', 'company_id', 'user_third_party_id', 'client_id', 'provider_id', 'request_id', 'client_payment_id', 'payment_currency', 'payment_date', 'total_amount_paid', 'capital_paid', 'interest_paid', 'guarantee_paid', 'last_status', 'last_paid_date', 'facturas_vencimientos_iguales', 'fecha_confirmada_hubspot', 'max_payment_date_invoices', 'e_payment_date', 'cambio_fecha_vencimiento', 'q_desembolso', 'm_desembolso', 'new_clients', 'recurrent_clients', 'new_providers', 'recurrent_providers', 'remaining_capital', 'remaining_total_amount', 'actual_status', 'flag_excluir', 'dias_atraso', 'm_desembolso_soles', 'remaining_capital_soles', 'amount_financed_soles', 'exchange_rate', 'codmes_transfer', 'PAR1_m', 'PAR15_m', 'PAR30_m', 'PAR60_m', 'PAR90_m', 'PAR120_m', 'PAR180_m', 'PAR360_m', 'PAR1_q', 'PAR15_q', 'PAR30_q', 'PAR60_q', 'PAR90_q', 'PAR120_q', 'PAR180_q', 'PAR360_q', 'q_vigente', 'condoned', 'judicialized', 'rango_dias_atraso', 'rango_duracion', 'PAR1_ms', 'PAR15_ms', 'PAR30_ms', 'PAR60_ms', 'PAR90_ms', 'PAR120_ms', 'PAR180_ms', 'PAR360_ms', 'FLAG_ORIGEN_OPERACION']
columnas_mayusc = [col.upper() for col in columnas]
cc = df_corte.columns
cc = [col.upper() for col in cc]
cv = df_view.columns
cv = [col.upper() for col in cv]

df_corte.columns = cc
df_corte = df_corte[columnas_mayusc]

df_view.columns = cv
df_view = df_view[columnas_mayusc]

df_corte.columns = columnas
df_view.columns  = columnas

df_concatenado = pd.concat([df_corte, df_view], ignore_index=True)
print(df_concatenado.shape)
print(df_concatenado['codmes'].unique())

(101731, 90)
[202012 202101 202102 202103 202104 202105 202106 202107 202108 202109
 202110 202111 202112 202201 202202 202203 202204 202205 202206 202207
 202208 202209 202210 202211 202212 202301 202302 202303 202304 202305
 202306 202307 202308 202309 202310 202311 202312 202401 202402 202403
 202404 202405 202406 202407 202408 202409 202410 202411 202412 202501
 202502 202503 202504 202505 202506 202507 202508 202509 202510 202511
 202512]


In [10]:
# bloque de código temporal para no incluir mes actual
if incluir_mes_actual == False:
  df_concatenado = df_concatenado[df_concatenado['codmes'] != int(codmes)]
else:
  print('')
print(df_concatenado['codmes'].unique())


[202012 202101 202102 202103 202104 202105 202106 202107 202108 202109
 202110 202111 202112 202201 202202 202203 202204 202205 202206 202207
 202208 202209 202210 202211 202212 202301 202302 202303 202304 202305
 202306 202307 202308 202309 202310 202311 202312 202401 202402 202403
 202404 202405 202406 202407 202408 202409 202410 202411 202412 202501
 202502 202503 202504 202505 202506 202507 202508 202509 202510 202511
 202512]


In [11]:
#%% CONVERTIR COLUMNAS DE FECHAS A UN SOLO FORMATO
n_alertas = 0
df_concatenado["fecha_cierre"] = pd.to_datetime(df_concatenado["fecha_cierre"],  format="mixed")
df_concatenado["fecha_cierre"] = df_concatenado["fecha_cierre"].dt.strftime("%Y-%m-%d")
alert = df_concatenado[ df_concatenado["fecha_cierre"].isna() ]
if alert.shape[0]>0:
    print('alerta de nulos en fecha_cierre')
    n_alertas += 1

df_concatenado["transfer_date"] = pd.to_datetime(df_concatenado["transfer_date"],  format="mixed")
df_concatenado["transfer_date"] = df_concatenado["transfer_date"].dt.strftime("%Y-%m-%d")
alert = df_concatenado[ df_concatenado["transfer_date"].isna() ]
if alert.shape[0]>0:
    print('alerta de nulos en transfer_date')
    n_alertas += 1

df_concatenado["e_payment_date_original"] = pd.to_datetime(df_concatenado["e_payment_date_original"],  format="mixed")
df_concatenado["e_payment_date_original"] = df_concatenado["e_payment_date_original"].dt.strftime("%Y-%m-%d")
alert = df_concatenado[ df_concatenado["e_payment_date_original"].isna() ]
if alert.shape[0]>0:
    print('alerta de nulos en e_payment_date_original')
    n_alertas += 1

df_concatenado["payment_date"] = pd.to_datetime(df_concatenado["payment_date"],  format="mixed")
df_concatenado["payment_date"] = df_concatenado["payment_date"].dt.strftime("%Y-%m-%d")
# alert = df_concatenado[ df_concatenado["payment_date"].isna() ]

df_concatenado["e_payment_date"] = pd.to_datetime(df_concatenado["e_payment_date"],  format="mixed")
df_concatenado["e_payment_date"] = df_concatenado["e_payment_date"].dt.strftime("%Y-%m-%d")
alert = df_concatenado[ df_concatenado["e_payment_date"].isna() ]
if alert.shape[0]>0:
    print('alerta de nulos en e_payment_date')
    n_alertas += 1

################################################################################
class StopExecution(Exception):
    pass
if n_alertas >= 1:
    raise StopExecution("Se canceló la ejecución por presencia de alertas")
else:
    print("No se encontraron alertas")
################################################################################


No se encontraron alertas


In [12]:
#%% obtener ruc del corte más reciente, para unirlo por codigo de subasta, a aquellos casos donde falte ruc
rucs = df_view_completo[['code', 'client_ruc', 'codmes']]
rucs = rucs[ ~pd.isna(rucs['client_ruc']) ]
rucs = rucs.sort_values(by = 'codmes', ascending = False)
rucs = rucs.drop_duplicates(subset=['code'], keep="first")

del rucs['codmes']
rucs.columns = ['code_aux', 'ruc_aux']

df_concatenado = df_concatenado.merge(rucs,
                                      left_on  = 'code',
                                      right_on = 'code_aux',
                                      how      = 'left')
df_concatenado['client_ruc'] = np.where( ~df_concatenado['ruc_aux'].isnull(),
                                         df_concatenado['ruc_aux'],
                                         df_concatenado['client_ruc'] )
del df_concatenado['code_aux']
del df_concatenado['ruc_aux']

In [13]:
#%% limpieza de texto del client_ruc y provider_ruc
col_ruc = 'client_ruc'
df_concatenado[col_ruc] = (
    df_concatenado[col_ruc]
    .astype(str)                 # convierte todo a string
    .str.replace('.0','', regex=False)  # limpia floats mal guardados
    .replace('None', pd.NA)      # convierte 'None' a valor nulo real
)
vr = df_concatenado[ (df_concatenado[col_ruc] == 'None') | (pd.isna(df_concatenado[col_ruc]))]

col_ruc = 'provider_ruc'
df_concatenado[col_ruc] = (
    df_concatenado[col_ruc]
    .astype(str)                 # convierte todo a string
    .str.replace('.0','', regex=False)  # limpia floats mal guardados
    .replace('None', pd.NA)      # convierte 'None' a valor nulo real
)
vr = df_concatenado[ (df_concatenado[col_ruc] == 'None') | (pd.isna(df_concatenado[col_ruc]))]

In [14]:
#%% parchamiento del nombre del client_ruc
clientes = df_concatenado[['client_ruc', 'client_name', 'codmes']]
clientes = clientes.sort_values(by='codmes', ascending=False)
clientes = clientes.drop_duplicates(subset=['client_ruc'], keep="first")
clientes = clientes[ ~pd.isna(clientes['client_ruc'])]
clientes = clientes[ ~pd.isna(clientes['client_name'])]
clientes.columns = ['client_ruc_aux', 'client_name_aux', 'codmes']

df_concatenado = df_concatenado.merge(clientes[['client_ruc_aux', 'client_name_aux']],
                                  left_on  = 'client_ruc',
                                  right_on = 'client_ruc_aux',
                                  how      = 'left')
df_concatenado['client_name'] = np.where( ~df_concatenado['client_name_aux'].isnull(),
                                         df_concatenado['client_name_aux'],
                                         df_concatenado['client_name'])
del df_concatenado['client_ruc_aux']
del df_concatenado['client_name_aux']


In [15]:
# colocando ruc en los casos faltantes
import pandas as pd
data = {
    'code': [
        'INACONS1.14.08.25',
        'ky1w5vqj',
        'jFfwjqbo',
        'Nxu9BM4H',
        'l836luog',
        'JVRESGUARDO.09.07.25'
    ],
    'rp_parcheo': [
        '20568587767',
        '20517228177',
        '20606024313',
        '20518791983',
        '20442847071',
        '20100901481'
    ],
    'rd_parcheo': [
        '20600058119',
        '20459349341',
        '20554556192',
        '20554556192',
        '20506453047',
        '20100113610'
    ]
}

df_faltantes = pd.DataFrame(data)


df_concatenado = df_concatenado.merge(df_faltantes,
                                      on = 'code',
                                      how = 'left')

df_concatenado['provider_ruc'] = np.where( df_concatenado['provider_ruc'].isna(),
                                          df_concatenado['rp_parcheo'],
                                          df_concatenado['provider_ruc'])

df_concatenado['client_ruc'] = np.where( df_concatenado['client_ruc'].isna(),
                                          df_concatenado['rd_parcheo'],
                                          df_concatenado['client_ruc'])

del df_concatenado['rp_parcheo']
del df_concatenado['rd_parcheo']

In [16]:
#%% Colocando el client y provider actual, de manera retroactiva, y modificando la enumeración de nuevo o recurrente

# df_concatenado = df_concatenado.sort_values(by = 'codmes', ascending = False) # este método solo actualiza a las ops que aparecen en el mes actual
# op_actual = df_concatenado[ df_concatenado['codmes'] == int(codmes) ]
df_view_completo = df_view_completo.sort_values(by = 'codmes', ascending = False)
df_view_completo['client_ruc'] = df_view_completo['client_ruc'].str.strip()
df_view_completo['provider_ruc'] = df_view_completo['provider_ruc'].str.strip()

actual_provider = df_view_completo[['code', 'codmes', 'provider_ruc', 'provider_name']].sort_values(by = 'codmes', ascending = False)
apl = actual_provider.drop_duplicates(subset=['provider_ruc'], keep="first")
apl = apl[['provider_ruc', 'provider_name']]
apl.columns = ['provider_ruc_aux', 'provider_name_aux']

actual_provider = actual_provider[ (actual_provider['provider_ruc'].notna()) & (actual_provider['code'].notna()) ]
actual_provider = actual_provider.drop_duplicates(subset=['code'], keep="first")
actual_provider = actual_provider[['code', 'provider_ruc']]
actual_provider.columns = ['code',  'provider_ruc_aux']
actual_provider = actual_provider.merge(apl, on = 'provider_ruc_aux', how = 'left')

actual_client = df_view_completo[['code', 'codmes', 'client_ruc', 'client_name']].sort_values(by = 'codmes', ascending = False)
acl = actual_client.drop_duplicates(subset=['client_ruc'], keep="first")
acl = acl[['client_ruc', 'client_name']]
acl.columns = ['client_ruc_aux', 'client_name_aux']

actual_client = actual_client[ (actual_client['client_ruc'].notna()) & (actual_client['code'].notna()) ]
actual_client = actual_client.drop_duplicates(subset=['code'], keep="first")
actual_client = actual_client[['code', 'client_ruc']]
actual_client.columns = ['code',  'client_ruc_aux']
actual_client = actual_client.merge(acl, on = 'client_ruc_aux', how = 'left')

df_concatenado = df_concatenado.merge(actual_provider,
                                      on = 'code',
                                      how = 'left')
df_concatenado = df_concatenado.merge(actual_client,
                                      on = 'code',
                                      how = 'left')

df_concatenado['client_ruc'] = np.where( df_concatenado['client_ruc_aux'].notna(),
                                         df_concatenado['client_ruc_aux'],
                                         df_concatenado['client_ruc'])
df_concatenado['client_name'] = np.where( df_concatenado['client_name_aux'].notna(),
                                          df_concatenado['client_name_aux'],
                                          df_concatenado['client_name'])
df_concatenado['provider_ruc'] = np.where( df_concatenado['provider_ruc_aux'].notna(),
                                           df_concatenado['provider_ruc_aux'],
                                           df_concatenado['provider_ruc'])
df_concatenado['provider_name'] = np.where( df_concatenado['provider_name_aux'].notna(),
                                            df_concatenado['provider_name_aux'],
                                            df_concatenado['provider_name'])
del df_concatenado['client_ruc_aux']
del df_concatenado['client_name_aux']
del df_concatenado['provider_ruc_aux']
del df_concatenado['provider_name_aux']

In [17]:
# de nuevo parchando el nombre del proveedor y deudor porque por alguna razón desconocida no funcionó en todos los casos
df_concatenado = df_concatenado.merge(apl,
                                      left_on  = 'provider_ruc',
                                      right_on = 'provider_ruc_aux',
                                      how      = 'left')
df_concatenado['provider_name'] = np.where( ~df_concatenado['provider_name_aux'].isnull(),
                                            df_concatenado['provider_name_aux'],
                                            df_concatenado['provider_name'])
del df_concatenado['provider_ruc_aux']
del df_concatenado['provider_name_aux']
################################################################################
df_concatenado = df_concatenado.merge(acl,
                                      left_on  = 'client_ruc',
                                      right_on = 'client_ruc_aux',
                                      how      = 'left')
df_concatenado['client_name'] = np.where( ~df_concatenado['client_name_aux'].isnull(),
                                            df_concatenado['client_name_aux'],
                                            df_concatenado['client_name'])
del df_concatenado['client_ruc_aux']
del df_concatenado['client_name_aux']


In [18]:
#%% Corrección de flag_newclient flag_newprovider, new_clients, recurrent_clients, new_providers, recurrent_providers
df_concatenado['transfer_date_aux'] = pd.to_datetime(df_concatenado['transfer_date'])
df_concatenado = df_concatenado.sort_values(by = ['codmes', 'transfer_date_aux'], ascending = [True,True])

# Creamos el contador acumulado para cada cliente
df_concatenado['flag_newclient'] = df_concatenado.groupby('client_ruc').cumcount() + 1
df_concatenado['flag_newclient'] = df_concatenado['flag_newclient'].fillna(0).astype(int)
df_concatenado['flag_newprovider'] = df_concatenado.groupby('provider_ruc').cumcount() + 1
df_concatenado['flag_newprovider'] = df_concatenado['flag_newprovider'].fillna(0).astype(int)

df_concatenado['new_clients'] = np.where(df_concatenado['flag_newclient'] == 1,
                                         1,
                                         0).astype(int)
df_concatenado['recurrent_clients'] = np.where(df_concatenado['flag_newclient'] == 1,
                                         0,
                                         1).astype(int)
df_concatenado['new_providers'] = np.where(df_concatenado['flag_newprovider'] == 1,
                                         1,
                                         0).astype(int)
df_concatenado['recurrent_providers'] = np.where(df_concatenado['flag_newprovider'] == 1,
                                         0,
                                         1).astype(int)

del df_concatenado['transfer_date_aux']


In [19]:
# recálculo de Terms hacia atras

df_terms = df_view_completo[['code', 'codmes', 'terms']].sort_values(by = 'codmes', ascending = False)
df_terms = df_terms.drop_duplicates(subset=['code'], keep="first")
df_terms = df_terms[['code', 'terms']]
df_terms.columns = ['code', 'terms_aux']

df_concatenado = df_concatenado.merge(df_terms,
                                      on = 'code',
                                      how = 'left')
df_concatenado['terms'] = np.where(df_concatenado['terms_aux'].notna(),
                                   df_concatenado['terms_aux'],
                                   df_concatenado['terms'])
del df_concatenado['terms_aux']


In [20]:
df_concatenado['codmes'].unique()

array([202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107,
       202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203,
       202204, 202205, 202206, 202207, 202208, 202209, 202210, 202211,
       202212, 202301, 202302, 202303, 202304, 202305, 202306, 202307,
       202308, 202309, 202310, 202311, 202312, 202401, 202402, 202403,
       202404, 202405, 202406, 202407, 202408, 202409, 202410, 202411,
       202412, 202501, 202502, 202503, 202504, 202505, 202506, 202507,
       202508, 202509, 202510, 202511, 202512])

In [21]:
# df_concatenado[ df_concatenado['provider_ruc']== '10423263542' ]

actual_provider[actual_provider['provider_ruc_aux'] == '20568587767']
# 10423263542

,code,provider_ruc_aux,provider_name_aux
2630,trHi1Vct,20568587767,INACONS SRL
2631,PXKrZh5S,20568587767,INACONS SRL
2632,vI0rOCbe,20568587767,INACONS SRL
2634,YP1ei9ac,20568587767,INACONS SRL
2635,l0U3zClw,20568587767,INACONS SRL
2641,1WpQ8cap,20568587767,INACONS SRL
2645,bdxCpAXN,20568587767,INACONS SRL
2646,cL9Ll9U4,20568587767,INACONS SRL
2647,r049Yzgh,20568587767,INACONS SRL
2684,sAKWo0V4,20568587767,INACONS SRL


In [22]:
#%%

# Cliente de S3
if cargar_sql == True:
  s3 = boto3.client(
      "s3",
      aws_access_key_id       = creds["AccessKeyId"],
      aws_secret_access_key   = creds["SecretAccessKey"],
      aws_session_token       = creds["SessionToken"],
      region_name             = creds["region_name"]
  )

  # ==== CONFIGURACIÓN ====
  bucket_name = "prod-datalake-raw-730335218320"
  s3_prefix = "manual/ba/fac_outstanding_monthly_snapshot/" # carpeta lógica en el bucket

  # ==== EXPORTAR A PARQUET EN MEMORIA ====
  csv_buffer = io.StringIO()
  df_concatenado.to_csv(csv_buffer, index=False, encoding="utf-8-sig")

  # Nombre de archivo con timestamp (opcional, para histórico)
  s3_key = f"{s3_prefix}fac_outstanding_monthly_snapshot.csv"

  # Subir directamente desde el buffer
  s3.put_object(Bucket = bucket_name,
                Key    = s3_key,
                Body   = csv_buffer.getvalue()
                )

  print(f"✅ Archivo subido a s3://{bucket_name}/{s3_key}")
else:
  print('archivo no cargado')

✅ Archivo subido a s3://prod-datalake-raw-730335218320/manual/ba/fac_outstanding_monthly_snapshot/fac_outstanding_monthly_snapshot.csv


In [23]:
print("Inicio:", inicio.strftime("%Y-%m-%d %H:%M:%S"))
print('')
fin = datetime.now()
print("Fin:", fin.strftime("%Y-%m-%d %H:%M:%S"))


Inicio: 2026-01-14 20:48:02

Fin: 2026-01-14 21:00:25
